In [1]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

from dmsensei import DataModule, create_model
from dmsensei.config import device
from dmsensei.core.callbacks import WandbFitLogger, KaggleLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import torch, wandb
import numpy as np

device = 'cuda'

# Create dataset
dm = DataModule(
    name=["ribo-kaggle"],
    data_type=["dms", "shape",'structure'],
    force_download=False,
    batch_size=32,
    num_workers=0,
    train_split=256,
    valid_split=0,
    predict_split=0,
    overfit_mode=True,
    shuffle_valid=False,
)

dm.setup('fit')

Loading dataset from disk...


In [2]:

import torch
import torch.nn.functional as F

lr = 5E-4
model = create_model(
    model="transformer",
    # weight_data=True,
    ntoken=5,
    d_model=64,
    nhead=16,
    d_hid=128,
    nlayers=2,
    dropout=0.0,
    lr=lr,
    weight_decay=0,
    gamma=0.997,
    c_z=8,
).to(device)

In [7]:
for batch in dm.train_dataloader():
    model.training_step(batch, 0)
    break

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [13]:
wandb.init(project="dmsensei", name='debug-structure')
trainer = Trainer(
    max_epochs=10000,
    callbacks=[
        WandbFitLogger(dm=dm)
    ],
    logger=WandbLogger(),
    )

trainer.fit(model, dm)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
train/loss,█▁▁
trainer/global_step,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
valid/dms/mae,█▇█▇▇▆▆▆▅▅▅▅▄▃▄▃▃▃▁▁▁▁
valid/dms/pearson,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
valid/dms/r2,▁▁▂▂▂▂▃▃▄▃▄▄▅▅▅▆▆▆▇▇██
valid/loss,██▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁
valid/loss_dms,██▇▇▇▇▆▆▅▆▅▅▄▄▄▃▃▃▂▂▁▁
valid/loss_shape,█▇▇▆▇▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▁▁
valid/loss_structure,██▆▆▄▄▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
valid/shape/mae,▅▄█▆▄▅▆▄▃▄▄▄▄▃▃▄▄▄▃▂▁▂


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                 | Type               | Params
------------------------------------------------------------
0 | encoder              | Embedding          | 320   
1 | pos_encoder          | PositionalEncoding | 0     
2 | encoder_adapter      | Linear             | 260   
3 | activ                | ReLU               | 0     
4 | transformer_encoder  | Sequential         | 66.9 K
5 | resnet               | Sequential         | 9.7 K 
6 | output_net_DMS       | Sequential         | 17.0 K
7 | output_net_SH

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.
/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.
/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
pred, true = batch.get_pairs('structure')